Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License [2017] Zalando SE, https://tech.zalando.com

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/work-with-data/datasets-tutorial/pipeline-with-datasets/pipeline-for-image-classification.png)

# Build a simple ML pipeline for image classification

## Introduction
This tutorial shows how to train a simple deep neural network using the [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) dataset and Keras on Azure Machine Learning. Fashion-MNIST is a dataset of Zalando's article imagesâ€”consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.

Learn how to:

> * Set up your development environment
> * Create the Fashion MNIST dataset
> * Create a machine learning pipeline to train a simple deep learning neural network on a remote cluster
> * Retrieve input datasets from the experiment and register the output model with datasets

## Prerequisite:
* Understand the [architecture and terms](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture) introduced by Azure Machine Learning
* If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, go through the [configuration notebook](../../../configuration.ipynb) to:
    * install the latest version of AzureML SDK
    * create a workspace and its configuration file (`config.json`)

## Set up your development environment

All the setup for your development work can be accomplished in a Python notebook.  Setup includes:

* Importing Python packages
* Connecting to a workspace to enable communication between your local computer and remote resources
* Creating an experiment to track all your runs
* Creating a remote compute target to use for training

### Import packages

Import Python packages you need in this session. Also display the Azure Machine Learning SDK version.

In [1]:
import os
import azureml.core
from azureml.core import Workspace, Datastore, ComputeTarget, RunConfiguration, Experiment
# from azureml.core import Dataset
from azureml.core.dataset import Dataset
from azureml.core.runconfig import CondaDependencies
from azureml.pipeline.steps import PythonScriptStep, EstimatorStep
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.train.dnn import TensorFlow

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.85


### Connect to workspace

Create a workspace object from the existing workspace. `Workspace.from_config()` reads the file **config.json** and loads the details into an object named `workspace`.

In [2]:
# load workspace
workspace = Workspace.from_config()
print('Workspace name: ' + workspace.name, 
      'Azure region: ' + workspace.location, 
      'Subscription id: ' + workspace.subscription_id, 
      'Resource group: ' + workspace.resource_group, sep='\n')

Workspace name: avadevitsmlsvc
Azure region: westus2
Subscription id: ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e
Resource group: RG-ITSMLTeam-Dev


### Create experiment and a directory

Create an experiment to track the runs in your workspace and a directory to deliver the necessary code from your computer to the remote resource.

In [3]:
# create an ML experiment
exp = Experiment(workspace=workspace, name='dataset-bug')

### Create or Attach existing compute resource
By using Azure Machine Learning Compute, a managed service, data scientists can train machine learning models on clusters of Azure virtual machines. Examples include VMs with GPU support. In this tutorial, you create Azure Machine Learning Compute as your training environment. The code below creates the compute clusters for you if they don't already exist in your workspace.

**Creation of compute takes approximately 5 minutes.** If the AmlCompute with that name is already in your workspace the code will skip the creation process.

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
# cluster_name = "your-cluster-name" # TODO CHANGE THIS DEMO COMPUTE TO BE DIFFERENT
cluster_name = "dataset-bug"

try:
    compute_target = ComputeTarget(workspace=workspace, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 1, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-01-31T03:20:26.715000+00:00', 'errors': None, 'creationTime': '2020-01-30T19:57:41.349092+00:00', 'modifiedTime': '2020-01-30T21:40:23.377992+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT600S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


In [5]:
# In the UI, register a dataset called 'Diabetes' using Azure Open Datasets
dataset_raw = Dataset.get_by_name(workspace, name='Diabetes')

dataset_raw

{
  "source": [
    "https://azureopendatastorage.blob.core.windows.net/mlsamples/diabetes/*.parquet"
  ],
  "definition": [
    "GetFiles",
    "ReadParquetFile"
  ],
  "registration": {
    "id": "28d89eeb-0052-47ae-9029-9b9d5cbfbf3d",
    "name": "Diabetes",
    "version": 1,
    "tags": {
      "opendatasets": "sample-diabetes"
    },
    "workspace": "Workspace.create(name='avadevitsmlsvc', subscription_id='ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e', resource_group='RG-ITSMLTeam-Dev')"
  }
}

In [6]:
datastore = workspace.get_default_datastore()
# define output data
output_dir = PipelineData('output_dir', datastore=datastore).as_dataset()

# register output data as dataset
output_dir = output_dir.register(name='output_dir', create_new_version=True)

## Build 2-step ML pipeline

The [Azure Machine Learning Pipeline](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-ml-pipelines) enables data scientists to create and manage multiple simple and complex workflows concurrently. A typical pipeline would have multiple tasks to prepare data, train, deploy and evaluate models. Individual steps in the pipeline can make use of diverse compute options (for example: CPU for data preparation and GPU for training) and languages. [Learn More](https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/machine-learning-pipelines)


### Step 1: data preparation

In step one, we will load the image and labels from Fashion MNIST dataset into mnist_train.csv and mnist_test.csv

Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255. Both mnist_train.csv and mnist_test.csv contain 785 columns. The first column consists of the class labels, which represent the article of clothing. The rest of the columns contain the pixel-values of the associated image.

In [15]:
conda = CondaDependencies(conda_dependencies_file_path='compute_dependencies_small.yml')

run_config = RunConfiguration()
run_config.environment.python.conda_dependencies = conda

Intermediate data (or output of a step) is represented by a `PipelineData` object. preprared_fashion_ds is produced as the output of step 1, and used as the input of step 2. PipelineData introduces a data dependency between steps, and creates an implicit execution order in the pipeline. You can register a `PipelineData` as a dataset and version the output data automatically. [Learn More](https://docs.microsoft.com/azure/machine-learning/service/how-to-version-track-datasets#version-a-pipeline-output-dataset) 

A **PythonScriptStep** is a basic, built-in step to run a Python Script on a compute target. It takes a script name and optionally other parameters like arguments for the script, compute target, inputs and outputs. If no compute target is specified, default compute target for the workspace is used. You can also use a [**RunConfiguration**](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.runconfiguration?view=azure-ml-py) to specify requirements for the PythonScriptStep, such as conda dependencies and docker image.

In [16]:
extract_step = PythonScriptStep(
    name='extract',
    script_name='extract.py',
    arguments=['--output_dir', output_dir,
               '--remote_run', True,
               #    '--n_rows', 10000
               ],
    compute_target=compute_target,
    inputs=[dataset_raw.as_named_input('dsIM')],
    outputs=[output_dir],
    runconfig=run_config,
    source_directory=os.path.join(os.getcwd(), 'extract'),
    allow_reuse=False
)

### Step 2: train CNN with Keras

Next, we construct an `azureml.train.dnn.TensorFlow` estimator object. The TensorFlow estimator is providing a simple way of launching a TensorFlow training job on a compute target. It will automatically provide a docker image that has TensorFlow installed.

[EstimatorStep](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-steps/azureml.pipeline.steps.estimator_step.estimatorstep?view=azure-ml-py) adds a step to run Tensorflow Estimator in a Pipeline. It takes a dataset as the input.

In [9]:
# # set up training step with Tensorflow estimator
# est = TensorFlow(entry_script='train.py',
#                  source_directory=script_folder,                 
#                  pip_packages = ['azureml-sdk','keras','numpy','scikit-learn', 'matplotlib'],
#                  compute_target=compute_target)

# est_step = EstimatorStep(name='train step',
#                          estimator=est,
#                          estimator_entry_script_arguments=[],
#                          # parse prepared_fashion_ds into TabularDataset and use it as the input
#                          inputs=[prepared_fashion_ds.parse_delimited_files()],
#                          compute_target=compute_target)

### Build the pipeline
Once we have the steps (or steps collection), we can build the [pipeline](https://docs.microsoft.com/python/api/azureml-pipeline-core/azureml.pipeline.core.pipeline.pipeline?view=azure-ml-py).

A pipeline is created with a list of steps and a workspace. Submit a pipeline using [submit](https://docs.microsoft.com/python/api/azureml-core/azureml.core.experiment(class)?view=azure-ml-py#submit-config--tags-none----kwargs-). When submit is called, a [PipelineRun](https://docs.microsoft.com/python/api/azureml-pipeline-core/azureml.pipeline.core.pipelinerun?view=azure-ml-py) is created which in turn creates [StepRun](https://docs.microsoft.com/python/api/azureml-pipeline-core/azureml.pipeline.core.steprun?view=azure-ml-py) objects for each step in the workflow.

In [10]:
# build pipeline & run experiment
pipeline = Pipeline(workspace,
                    steps=[
                        extract_step,
#                         est_step
                    ])

In [11]:
run = exp.submit(pipeline)

Created step extract [f2ce22f3][e1313719-99e0-4b69-b31c-2bce6f888826], (This step will run and generate new outputs)
Submitted PipelineRun eaf8ec36-5c3f-4815-94a2-6cc18199d234
Link to Azure Machine Learning studio: https://ml.azure.com/experiments/dataset-bug/runs/eaf8ec36-5c3f-4815-94a2-6cc18199d234?wsid=/subscriptions/ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e/resourcegroups/RG-ITSMLTeam-Dev/workspaces/avadevitsmlsvc


In [13]:
run.tag("Dataset","Diabetes")
run.tag("pipeline", "reprex")

### Monitor the PipelineRun

In [14]:
run.wait_for_completion(show_output=True)

PipelineRunId: eaf8ec36-5c3f-4815-94a2-6cc18199d234
Link to Portal: https://ml.azure.com/experiments/dataset-bug/runs/eaf8ec36-5c3f-4815-94a2-6cc18199d234?wsid=/subscriptions/ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e/resourcegroups/RG-ITSMLTeam-Dev/workspaces/avadevitsmlsvc
PipelineRun Status: Running


StepRunId: 31e87ce2-2b8d-4026-b097-fd25c2ccc331
Link to Portal: https://ml.azure.com/experiments/dataset-bug/runs/31e87ce2-2b8d-4026-b097-fd25c2ccc331?wsid=/subscriptions/ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e/resourcegroups/RG-ITSMLTeam-Dev/workspaces/avadevitsmlsvc
StepRun( extract ) Status: Running

Streaming azureml-logs/55_azureml-execution-tvmps_d50610a5f50e669eef13c30ca7eca9bef278aebd8a3cbe9427f2eef91c3021e9_d.txt
2020-01-31T03:33:06Z Starting output-watcher...
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_5e7bdd97db81db832946e9315e2ae09e
Digest: sha256:698796d7eef3522a9e7d0b3a946107aad4b3f98c4fd3dc778908544bdcba7e88
Status: Image is up to date for avade

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "User program failed with KeyError: 'dsIM'",
        "detailsUri": "https://aka.ms/azureml-known-errors",
        "details": [],
        "debugInfo": {
            "type": "KeyError",
            "message": "'dsIM'",
            "stackTrace": "  File \"/mnt/batch/tasks/shared/LS_root/jobs/avadevitsmlsvc/azureml/31e87ce2-2b8d-4026-b097-fd25c2ccc331/mounts/workspaceblobstore/azureml/31e87ce2-2b8d-4026-b097-fd25c2ccc331/azureml-setup/context_manager_injector.py\", line 119, in execute_with_context\n    runpy.run_path(sys.argv[0], globals(), run_name=\"__main__\")\n  File \"/azureml-envs/azureml_182fc27bcb4bbd15b484e4b20be151a5/lib/python3.6/runpy.py\", line 263, in run_path\n    pkg_name=pkg_name, script_name=fname)\n  File \"/azureml-envs/azureml_182fc27bcb4bbd15b484e4b20be151a5/lib/python3.6/runpy.py\", line 96, in _run_module_code\n    mod_name, 